In [1]:
import os

# Import docx
import docx
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx import Document
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.shared import Cm
from docx.shared import Pt

# Import QRCode from pyqrcode 
import pyqrcode 
import png 
from pyqrcode import QRCode

# Import Pillow
from PIL import Image, ImageOps
from PIL import ImageFont
from PIL import ImageDraw 

In [2]:
def set_repeat_table_header(row):
    """ set repeat table row on every new page
    """
    tr = row._tr
    trPr = tr.get_or_add_trPr()
    tblHeader = OxmlElement('w:tblHeader')
    tblHeader.set(qn('w:val'), "true")
    trPr.append(tblHeader)
    return row

def tight_margin(doc):

    current_section = doc.sections[-1]
    current_section.top_margin = Cm(1.4)
    current_section.bottom_margin = Cm(1.4)
    current_section.left_margin = Cm(2)
    current_section.right_margin = Cm(2)

In [3]:
def generate_qr_code(directory, s):
    
    # Generate the QR code 
    url = pyqrcode.create(s)
    
    # Generate image filename
    filename = os.path.join(directory, s+".png")
    
    # Create and save the png file naming "myqr.png" 
    url.png(filename, scale = 3)
    
    img = Image.open(filename)
    rgbimg = img.convert('RGB')
    W, H = rgbimg.size
    draw = ImageDraw.Draw(rgbimg)
    # Add text
    msg = "{}".format(s)
    font = ImageFont.truetype("FONTS/arialbd.ttf", 11)
    w, h = draw.textsize(msg, font=font)
    draw.text(((W+1-w)/2,H-11), msg, font=font, fill="#00c1e4")
    rgbimg.save(filename)

In [4]:
def generate_qr_codes_2prow(directory, df, n0, n1, country):
    
    for _, row in df.iterrows():
        
        f = row['name']
        fname = row['label']
        
        word_doc = 'TIMCI {0} - F{1:04d} {2} - Participants {3} to {4}.docx'.format(country, f, fname, n0, n1)
        
        # Create a new Word document
        doc = Document()
        doc.styles['Normal'].font.name = "Segoe UI"
        doc.styles['Normal'].font.size = Pt(10)
        tight_margin(doc)
        
        # add a table to the end and create a reference variable
        # extra row is so we can add the header row
        tab = doc.add_table(rows=int((n1-n0+1)/2)+1, cols=4)
        set_repeat_table_header(tab.rows[0])
        
        # Add header
        #section = doc.sections[0]
        #header = section.header
        #paragraph = header.paragraphs[0]
        #paragraph.text = 'TIMCI {0} - F{1:04d} {2}'.format(country, f, fname)

        # add the header rows.
        tab.cell(0,0).text = "Autocollant à coller dans le carnet de santé"
        paragraph = tab.cell(0,0).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tab.cell(0,1).text = "Autocollant à coller dans le registre"
        paragraph = tab.cell(0,1).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tab.cell(0,2).text = "Autocollant à coller dans le carnet de santé"
        paragraph = tab.cell(0,2).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
        tab.cell(0,3).text = "Autocollant à coller dans le registre"
        paragraph = tab.cell(0,3).paragraphs[0]
        paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

        # add rows
        for p in range(n0-n0+1, int((n1-n0+1)/2)+1):

            # String which represents the QR code 
            s1 = "{0}-F{1:04d}-P{2:04d}".format(country[0], f, n0-1+2*p-1)

            # Generate QR code 
            url = pyqrcode.create(s1)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s1+".png"), scale = 2)
            
            infile1 = os.path.join(os.path.join(directory, s1+".png"))
            
            paragraph = tab.cell(p,0).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,0).add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,0).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            paragraph = tab.cell(p,1).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,1).add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,1).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            # String which represents the QR code 
            s2 = "{0}-F{1:04d}-P{2:04d}".format(country[0], f, n0-1+2*p)

            # Generate QR code 
            url = pyqrcode.create(s2)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s2+".png"), scale = 2)
            
            infile2 = os.path.join(os.path.join(directory, s2+".png"))
            
            paragraph = tab.cell(p,2).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile2)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,2).add_paragraph(s2)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,2).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            paragraph = tab.cell(p,3).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile2)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,3).add_paragraph(s2)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,3).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
        # save the doc
        doc.save(os.path.join(directory, word_doc))

In [5]:
import pandas as pd
df = pd.read_csv(r"C:\Users\langhe\switchdrive\Institution\TIMCI\1 Project details\11 DM\3 ODK\odk_forms\05 Senegal\1-RCT-LS\01a-TIMCI-CRF-Facility-Senegal-form\facilities.csv", sep=",")
df['name'] = df.apply(lambda row: int(row['name'].replace('F','')), axis=1)
print(df[['name','label']])

    name                      label
0      9                 PS Caritas
1     40               PS Cite Lamy
2     57         PS Darou Khoudouss
3     15           PS Diamaguene II
4     58                  PS Dioggo
5     10         PS Elisabeth Diouf
6     44             PS Grand Thiès
7     45                 PS Kawsara
8     59          PS Keur Mbir Ndao
9     37              PS Louly Ndia
10    19  PS Malicounda Keur Maïssa
11    60              PS Mbayenne 3
12    21        PS Mbour Toucouleur
13    22                  PS Medine
14    23              PS Ngaparou I
15    27          PS Pointe Sarrène
16    39                PS Sandiara
17    30               PS Santessou
18    53               PS Takhi Kao
19    34                  PS Varedo
20    14             PS Darou Salam
21    31                 PS Santhie
22    12               PS Ngueniène
23    38              PS Ndiaganiao


In [6]:
df = df[df['name'].isin([15,21])]
df

,name,label,type,category,ccode,lvl1,lvl2,intervention,spa,latitude,longitude
3,15,PS Diamaguene II,health post,urban,S,Thiès,Mbour,0,0,14.42597,-16.96631
12,21,PS Mbour Toucouleur,health post,urban,S,Thiès,Mbour,0,0,14.40372,-16.95574


In [7]:
generate_qr_codes_2prow(r'QR_codes_Senegal', df, 2001, 2500, "Senegal")

In [20]:
def generate_hcpqr_codes_2prow(directory, df, n):
    
    for _, row in df.iterrows():
        
        f = row['name']
        fname = row['label']
        
        word_doc = 'TIMCI Senegal - F{0:04d} {1} - Providers 1 to {2}.docx'.format(f, fname, n)
        
        # Create a new Word document
        doc = Document()
        doc.styles['Normal'].font.name = "Segoe UI"
        doc.styles['Normal'].font.size = Pt(10)
        tight_margin(doc)
        
        # add a table to the end and create a reference variable
        # extra row is so we can add the header row
        tab = doc.add_table(rows=n, cols=2)
        
        # add rows
        for p in range(1, n):

            # String which represents the QR code 
            s1 = "S-F{0:04d}-H{1:02d}".format(f, n)

            # Generate QR code 
            url = pyqrcode.create(s1)

            # Create and save the png file naming "myqr.png" 
            url.png(os.path.join(directory, s1+".png"), scale = 2)
            
            infile1 = os.path.join(os.path.join(directory, s1+".png"))
            
            paragraph = tab.cell(p,0).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            run = paragraph.add_run()
            pic = run.add_picture(infile1)
            paragraph.paragraph_format.space_after = Pt(0)
            paragraph = tab.cell(p,0).add_paragraph(s1)
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph.paragraph_format.space_after = Pt(3)
            paragraph = tab.cell(p,0).add_paragraph("Date: ____ / ____ / _______")
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            
            paragraph = tab.cell(p,1).paragraphs[0]
            paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER
            paragraph = tab.cell(p,1).add_paragraph("First name: ___________________________________")
            paragraph = tab.cell(p,1).add_paragraph("Last name: ____________________________________")
            paragraph = tab.cell(p,1).add_paragraph("Position: _____________________________________")
            
        # save the doc
        doc.save(os.path.join(directory, word_doc))

In [10]:
generate_hcpqr_codes_2prow(r'QR_codes_Senegal', df, 10)